<a href="https://colab.research.google.com/github/PaulaFlavia/ML--transferLearning/blob/main/Transfer_learning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importar bibliotecas
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

# URL do dataset
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

# Carregar dataset de gatos e cachorros
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=url, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  os.path.join(PATH, 'train'),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(224, 224),
  batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  os.path.join(PATH, 'train'),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(224, 224),
  batch_size=32)

# Pré-processamento das imagens
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Carregar modelo pré-treinado
base_model = keras.applications.VGG16(
    weights='imagenet',  # Carregar pesos pré-treinados
    input_shape=(224, 224, 3),
    include_top=False)  # Não incluir camadas densas no topo

# Congelar camadas convolucionais
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas densas
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compilar e treinar modelo
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)

# Avaliar modelo
model.evaluate(val_ds)

# Salvar modelo treinado
model.save('cats_vs_dogs.h5')



Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.
58889256/58889256 [==============================] - 0s 0us/step


Epoch 1/10
50/50 [==============================] - 832s 17s/step - loss: 1.2365 - accuracy: 0.7475 - val_loss: 0.2552 - val_accuracy: 0.8900
Epoch 2/10
50/50 [==============================] - 877s 18s/step - loss: 0.2101 - accuracy: 0.9069 - val_loss: 0.2229 - val_accuracy: 0.8950
Epoch 3/10
50/50 [==============================] - 869s 17s/step - loss: 0.1654 - accuracy: 0.9331 - val_loss: 0.2156 - val_accuracy: 0.9075
Epoch 4/10
50/50 [==============================] - 832s 17s/step - loss: 0.1268 - accuracy: 0.9581 - val_loss: 0.2060 - val_accuracy: 0.9100
Epoch 5/10
50/50 [==============================] - 831s 17s/step - loss: 0.0877 - accuracy: 0.9675 - val_loss: 0.1980 - val_accuracy: 0.9150
Epoch 6/10
50/50 [==============================] - 836s 17s/step - loss: 0.0667 - accuracy: 0.9775 - val_loss: 0.2116 - val_accuracy: 0.9125
Epoch 7/10
50/50 [==============================] - 831s 17s/step - loss: 0.0552 - accuracy: 0.9856 - val_loss: 0.2038 - val_accuracy: 0.9125
Epoch 